In [0]:
import json
import os

In [0]:
%run ../Utils/initialize

##<img src="https://databricks.com/wp-content/themes/databricks/assets/images/header_logo_2x.png" alt="logo" width="150"/> 

## SAT Check Configuration
* <i> This utility helps administrators review their current configuration of SAT checks and modify if needed. </i>
* <b> get_all_sat_checks() </b> 
  * Populates 'SAT Check' drop down widget with all relevant checks for the current cloud type & reset all other widgets
  * Administrator chooses a check from the dropdown list
* <b> get_sat_check_config() </b> 
  * Fetches the specific check details from the database for user to view or modify
* <b> set_all_sat_checks() </b> 
  * Updates Database with user changes to check details

In [0]:
%run ./../Utils/sat_checks_config

In [0]:
args = dbutils.notebook.entry_point.getCurrentBindings()

In [0]:
user_input = args['user_inputs']
user_enable = args['user_enables']
user_evaluate = args['user_evaluate']
user_alert = args['user_alert']

print(user_input,user_enable,user_evaluate,user_alert)

In [0]:
#Determine cloud type
def getCloudType(url):
  if '.cloud.' in url:
    return 'aws'
  elif '.azuredatabricks.' in url:
    return 'azure'
  elif '.gcp.' in url:
    return 'gcp'
  return ''

def getConfigPath():
  import os
  cwd = os.getcwd().lower()
  if (cwd.rfind('/includes') != -1) or (cwd.rfind('/setup') != -1) or (cwd.rfind('/utils') != -1):
    return '../../configs'
  elif (cwd.rfind('/notebooks') != -1):
    return '../configs'
  else:
    return 'configs'

In [0]:
#Reset SAT check widgets 
def get_all_sat_checks():
    dbutils.widgets.removeAll()

    hostname = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().getOrElse(None)
    cloud_type = getCloudType(hostname)

    s_sql = '''
        SELECT CONCAT_WS('_',check_id, category, check) AS check
        FROM {analysis_schema_name}.security_best_practices 
        WHERE {cloud_type}=1
        '''.format(cloud_type = cloud_type, analysis_schema_name= json_["analysis_schema_name"])

    all_checks = spark.sql(s_sql)
    checks = all_checks.rdd.map(lambda row : row[0]).collect()

    checks.sort()
    #print(checks)
    first_check = str(user_input)
    
    #Define Driver Widgets
    dbutils.widgets.dropdown("sat_check", first_check, [str(x) for x in checks], "a. SAT Check")

In [0]:
#Populate 'SAT Check' drop down widget with all relevant checks for the current cloud type & reset all other widgets
get_all_sat_checks()

In [0]:
def get_sat_check_config():
    sat_check = dbutils.widgets.get("sat_check")
    check_id = sat_check.split('_')[0]

    s_sql = '''
                SELECT enable, evaluation_value, alert
                FROM {analysis_schema_name}.security_best_practices 
                WHERE check_id= '{check_id}'
            '''.format(check_id = check_id, analysis_schema_name= json_["analysis_schema_name"])

    get_check = spark.sql(s_sql)
    check = get_check.toPandas().to_dict(orient = 'list')
    
#    enable = check['enable'][0]
#    evaluate = check['evaluation_value'][0]
#    alert = check['alert'][0]
    enable = user_enable
    evaluate = user_evaluate
    alert = user_alert


    dbutils.widgets.dropdown("check_enabled", str(enable),  ['0','1'], "b. Check Enabled")
    dbutils.widgets.text("evaluation_value", str(evaluate), "c. Evaluation Value")
    dbutils.widgets.text("alert", str(alert), "d. Alert")

In [0]:
#Fetch specific check details from the database for user to view or modify
get_sat_check_config()

In [0]:
#Update Database with user changes to check details
set_sat_check_config()
#reset widgets
get_all_sat_checks()